In [ ]:
from matplotlib import pyplot
import numpy
from scipy.sparse import coo_matrix

import pyibm

%matplotlib inline

In [ ]:
pyibm.__version__

In [ ]:
delta_kernel = pyibm.delta_peskin_2002
delta_kernel_size = 4

In [ ]:
def print_matrix_info(Op):
    print('Type: ', type(Op))
    print('Shape: ', Op.shape)
    print('Size: ', Op.data.size)
    print('Min/Max: ', Op.data.min(), Op.data.max())

In [ ]:
def plot_matrix(M, figsize=(6.0, 6.0), markersize=1):
    """Plot non-zero structure of an operator."""
    if not isinstance(M, coo_matrix):
        M = coo_matrix(M)
    pyplot.rc('font', family='serif', size=16)
    fig, ax = pyplot.subplots(figsize=figsize)
    ax.scatter(M.col, M.row, c='red', s=markersize, marker='s')
    ax.axis('scaled', adjustable='box')
    ax.set_xlim(0, M.shape[1] - 1)
    ax.set_ylim(0, M.shape[0] - 1)
    ax.invert_yaxis()
    ax.set_xticks([])
    ax.set_yticks([])
    return fig, ax

In [ ]:
def plot_contourf(field, grid, body=None,
                  levels=None,
                  axis_lim=(None, None, None, None),
                  show_grid=False):
    """Plot the filled contour of the 2D field."""
    pyplot.rc('font', family='serif', size=16)
    fig, ax = pyplot.subplots(figsize=(8.0, 8.0))
    X, Y = numpy.meshgrid(grid.x.vertices, grid.y.vertices)
    if show_grid:
        xmin, xmax = grid.x.start, grid.x.end
        ymin, ymax = grid.y.start, grid.y.end
        for xi in grid.x.vertices:
            ax.axvline(xi, ymin=ymin, ymax=ymax, color='grey')
        for yi in grid.y.vertices:
            ax.axhline(yi, xmin=xmin, xmax=xmax, color='grey')
        ax.scatter(X, Y, marker='x')
    if body is not None:
        ax.scatter(body.x, body.y, color='C3')
    if levels is None:
        levels = numpy.linspace(numpy.min(field), numpy.max(field), num=51)
    contf = ax.contourf(X, Y, field,
                        levels=levels, extend='both', zorder=0)
    fig.colorbar(contf)
    ax.axis('scaled', adjustable='box')
    ax.axis(axis_lim)
    return fig, ax

In [ ]:
def plot_grids(grid, body=None, Op=None,
               axis_lim=(None, None, None, None)):
    """Plot the 2D grid."""
    pyplot.rc('font', family='serif', size=16)
    fig, ax = pyplot.subplots(figsize=(8.0, 8.0))
    xmin, xmax = grid.x.start, grid.x.end
    ymin, ymax = grid.y.start, grid.y.end
    for xi in grid.x.vertices:
        ax.axvline(xi, ymin=ymin, ymax=ymax, color='grey')
    for yi in grid.y.vertices:
        ax.axhline(yi, xmin=xmin, xmax=xmax, color='grey')
    gridc = pyibm.GridCellCentered(grid=grid)
    X, Y = numpy.meshgrid(gridc.x.vertices, gridc.y.vertices)
    ax.scatter(X, Y, label='cell-centered', marker='o', s=20)
    gridx = pyibm.GridFaceX(grid=grid)
    X, Y = numpy.meshgrid(gridx.x.vertices, gridx.y.vertices)
    ax.scatter(X, Y, label='x-face', marker='x', s=20)
    gridy = pyibm.GridFaceY(grid=grid)
    X, Y = numpy.meshgrid(gridy.x.vertices, gridy.y.vertices)
    ax.scatter(X, Y, label='y-face', marker='x', s=20)
    if body is not None:
        ax.scatter(body.x, body.y, label='body', s=40)
        neighbors = body.get_neighbors(gridc)
        xn, yn = [], []
        for xb, yb, neighbor in zip(body.x, body.y, neighbors):
            i, j, _ = gridc.ijk(neighbor)
            xi, yj = gridc.x.vertices[i], gridc.y.vertices[j]
            xn.append(xi)
            yn.append(yj)
            ax.plot([xb, xi], [yb, yj], color='black')
        ax.scatter(xn, yn, label='neighbors',
                   marker='*', s=40, color='black')
    if Op is not None:
        Op = coo_matrix(Op)
        rows = Op.row
        cols = Op.col
        for row, col in zip(rows, cols):
            if row % body.ndim == 0:
                i, j, _ = gridx.ijk(col)
                xi, yj = gridx.x.vertices[i], gridx.y.vertices[j]
                ax.scatter(xi, yj, marker='s', s=40, color='navy')
        
#     ax.legend()
    ax.axis('scaled', adjustable='box')
    ax.axis(axis_lim)
    return fig, ax

In [ ]:
config = dict(x=dict(start=-2.0, end=2.0, num_cells=20),
              y=dict(start=-2.0, end=2.0, num_cells=20))
grid = pyibm.GridBase(config=config)
grid

In [ ]:
gridc = pyibm.GridCellCentered(grid=grid)
gridx = pyibm.GridFaceX(grid=grid)
gridy = pyibm.GridFaceY(grid=grid)
print(gridc)
print(gridx)
print(gridy)

In [ ]:
GHat = pyibm.assemble_GHat(gridc, gridx, gridy)
print_matrix_info(GHat)
plot_matrix(GHat);

In [ ]:
MHat = pyibm.assemble_MHat(gridx, gridy)
print_matrix_info(MHat)
plot_matrix(MHat, figsize=(3.0, 3.0));

In [ ]:
G = MHat @ GHat
print_matrix_info(G)
plot_matrix(G);

In [ ]:
DHat = pyibm.assemble_DHat(gridc, gridx, gridy)
print_matrix_info(DHat)
plot_matrix(DHat);

In [ ]:
R = pyibm.assemble_R(gridx, gridy)
print_matrix_info(R)
plot_matrix(R, figsize=(3.0, 3.0));

In [ ]:
RInv = pyibm.assemble_RInv(R)
print_matrix_info(RInv)
plot_matrix(RInv, figsize=(3.0, 3.0));

In [ ]:
D = DHat @ RInv
print_matrix_info(D)
plot_matrix(D);

In [ ]:
# Check if divergence of the opposite of the transpose of the gradient.
A = D + G.T
A = A.multiply(A > 1e-12)
assert A.nnz == 0

In [ ]:
radius = 0.5
xc, yc = 0.0, 0.0
ds = gridx.x.get_widths()[0]
N = int(round(2 * numpy.pi * radius / ds))
epsilon = 0.0
theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1] + epsilon
x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
body = pyibm.Body(x, y, grid=gridc)
body

In [ ]:
Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                          kernel=delta_kernel,
                          kernel_size=delta_kernel_size)
print_matrix_info(Op)
plot_matrix(Op);

In [ ]:
S = pyibm.assemble_surfaces(body)
print_matrix_info(S)
plot_matrix(S, figsize=(3.0, 3.0));

In [ ]:
E = Op @ R @ MHat
print_matrix_info(E)
plot_matrix(E);

In [ ]:
H = Op.T @ S
print_matrix_info(H)
plot_matrix(H);

In [ ]:
ux = numpy.ones(gridx.size)
uy = numpy.zeros(gridy.size)
u = numpy.concatenate((ux, uy))
Ub = E @ u
print(Ub.shape)
print(Ub)

In [ ]:
Fx = numpy.ones(body.size)
Fy = numpy.zeros(body.size)
F = numpy.empty(body.ndim * body.size)
F[::body.ndim], F[1::body.ndim] = Fx, Fy

f = H @ F
fx = f[:gridx.size].reshape(gridx.shape)
fy = f[gridx.size:].reshape(gridy.shape)

F2 = E @ f
Fx2, Fy2 = F2[::body.ndim], F2[1::body.ndim]

dS = body.ds
dx = gridc.x.get_widths()[gridc.ijk(body.neighbors[0])[0]]
dy = gridc.y.get_widths()[gridc.ijk(body.neighbors[0])[1]]
dV = dx * dy

print(numpy.sum(Fx) * dS, numpy.sum(Fy) * dS)
print(numpy.sum(fx) * dV, numpy.sum(fy) * dV)
print(numpy.sum(Fx2) * dS, numpy.sum(Fy2) * dS)

In [ ]:
plot_contourf(fx, gridx, body, show_grid=False);

In [ ]:
axis_lim = (-1.0, 1.0, -1.0, 1.0)
plot_grids(grid, body=body, axis_lim=axis_lim);

In [ ]:
Re = 100.0
L = 1 / Re * pyibm.assemble_L(gridx, gridy)
print_matrix_info(L)
plot_matrix(L);

In [ ]:
MHatInv = pyibm.assemble_MHatInv(MHat)
MInv = R @ MHatInv
dt = 0.01
BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=3, L=L, MInv=MInv)
print_matrix_info(BN)
plot_matrix(BN);

In [ ]:
EBNH = E @ BN @ H
print_matrix_info(EBNH)
plot_matrix(EBNH);

In [ ]:
numpy.linalg.cond(EBNH.todense())

In [ ]:
Nmax = 3
for N in range(1, Nmax + 1):
    BN_test = pyibm.assemble_BN(gridx, gridy, dt=dt,
                                N=N, L=L, MInv=MInv)
    DBNG_test = D @ BN_test @ G
    cond = numpy.linalg.cond(DBNG_test.todense())
    print(N, cond)

In [ ]:
Nmax = 3
for N in range(1, Nmax + 1):
    BN_test = pyibm.assemble_BN(gridx, gridy, dt=dt,
                                N=N, L=L, MInv=MInv)
    EBNH_test = E @ BN_test @ H
    cond = numpy.linalg.cond(EBNH_test.todense())
    print(N, cond)

In [ ]:
for r in [1.0, 2.0, 4.0]:
    ds = gridx.x.get_widths()[0] / r
    Nb = int(round(2 * numpy.pi * radius / ds))
    theta = numpy.linspace(0.0, 2 * numpy.pi, num=Nb + 1)[:-1]
    x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
    body_test = pyibm.Body(x, y, grid=gridc)
    Op_test = pyibm.assemble_delta(body_test, gridc, gridx, gridy,
                                   kernel=delta_kernel,
                                   kernel_size=delta_kernel_size)
    S_test = pyibm.assemble_surfaces(body_test)
    E_test = Op_test @ R @ MHat
    H_test = Op_test.T @ S_test
    Nmax = 3
    for N in range(1, Nmax + 1):
        BN_test = pyibm.assemble_BN(gridx, gridy, dt=dt,
                                    N=N, L=L, MInv=MInv)
        EBNH_test = E_test @ BN_test @ H_test
        cond = numpy.linalg.cond(EBNH_test.todense())
        print(r, N, cond)